In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

def calculate_alpha_beta(tickers, weights, benchmark, start_date, end_date):
    # Fetch data from Yahoo Finance API
    portf_returns = pd.DataFrame()
    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date)['Adj Close'].pct_change()
        print(f'YF data size: {ticker}: {data.size}')
        print(f'Volatility of {ticker}: {np.std(data) * np.sqrt(252):.4f}')
        portf_returns[ticker] = data

    benchmark_returns = yf.download(benchmark, start=start_date, end=end_date)['Adj Close'].pct_change()
    print(f'Volatility of {benchmark}: {np.std(benchmark_returns) * np.sqrt(252):.4f}')

    # Omit the first row which may contain NaN values
    portf_returns = portf_returns.iloc[1:]
    benchmark_returns = benchmark_returns.iloc[1:]

    # Calculate portfolio returns
    portf_returns['Portfolio'] = np.dot(portf_returns, weights)

    # Calculate covariance of portfolio and benchmark returns
    cov_matrix = np.cov(portf_returns['Portfolio'], benchmark_returns)

    # Calculate beta
    beta = cov_matrix[0, 1] / benchmark_returns.var()

    # Calculate alpha
    alpha = (portf_returns['Portfolio'].mean() - beta * benchmark_returns.mean())

    portf_stdev = portf_returns['Portfolio'].std()

    return alpha, beta, portf_returns['Portfolio'].mean(), portf_stdev

In [ ]:
# tickers = ['ADBE', 'AMAT', 'ARKK', 'BLDR', 'BRK-B', 'ETHE', 'FTNT', 'GBTC', 'QQQE', 'VXX', 'TMF', 'TSLA']
# weights = [0.075, 0.075, 0.1, 0.1, 0.125, 0.05, 0.075, 0.05, 0.125, -0.125, 0.1, 0.075]
tickers = ["ACLS", "AMPH", "AMR", "ANF", "APP", "ARCH", "ASC", "BXC", "CAAP", "CAH", "CLS", "CMCSA", "COP", "CRM", "CVX", "DINO", "GOOGL", "GRBK", "HLIT", "JXN", "META", "MFC", "MHO", "MOD", "MPC", "NUE", "OKTA", "PEP", "PERI", "POWL", "SMCI", "STRL", "SU", "TEX", "TGLS", "TMUS", "TWLO", "UBER", "URI", "VLO", "XOM"]
nTickers = len(tickers)
weights = [1.0 / nTickers] * nTickers # equal weight
benchmark = 'QQQ'
start_date = '2022-07-01'
end_date = '2024-03-31'

alpha, beta, portfolio_return, portfolio_volatility = calculate_alpha_beta(tickers, weights, benchmark, start_date, end_date)

print(f'Alpha: {alpha*252:.4f}')
print(f'Beta: {beta:.4f}')
print(f'Portfolio Return: {portfolio_return*252:.4f}')
print(f'Portfolio Volatility: {portfolio_volatility*np.sqrt(252):.4f}')

In [ ]:
import yfinance as yf
import pandas as pd

tickers = ["SPY", "ACLS", "AMPH", "AMR", "ANF", "APP", "ARCH", "ASC", "BXC", "CAAP", "CAH", "CLS", "CMCSA", "COP", "CRM", "CVX", "DINO", "GOOGL", "GRBK", "HLIT", "JXN", "LTHM", "META", "MFC", "MHO", "MOD", "MPC", "NUE", "OKTA", "PEP", "PERI", "POWL", "SMCI", "STRL", "SU", "TA", "TEX", "TGLS", "TMUS", "TWLO", "UBER", "URI", "VLO", "XOM"]
start_date = "2021-01-01"
data = pd.DataFrame()

for ticker in tickers:
    tmp = yf.download(ticker, start=start_date)
    tmp = tmp[["Adj Close"]]
    tmp.columns = [ticker]
    if data.empty:
        data = tmp
    else:
        data = data.join(tmp)

data.to_csv("D:\Temp\stocks_data.csv")